# INFO 371 Lab 9: Image Detection

#### Instructions

Please submit your completed lab notebook by the deadline. Working together is fun and useful but you must submit your own work. Discussing the solutions and problems with your instructors and classmates is completely fine. However, **do not** copy and paste their solution(s). Remember - your code/work should not appear (directly or indirectly) on any one else's machine and vice versa. 


#### Introduction
For this lab, you will be doing some image recognition, much like in Problem Set 4.

### Part 1: Data preparation

To start, let's prepare the data. This will have several steps.

#### 1. Examine the filenames

Download the squares-circles-crosses.zip file and extract it on your machine. This folder contains 6,000 greyscale images that are 32x32 pixels each. The data is already split into training and test (validation) sets using a 80/20 split. The images depict squares, circles, or crosses and the file names indicate which is in each image - "sq" for square, "ci" for circle", and "cr" for crosses. How many files are in the training data and how many are in the test data?

We have 4,800 files in the training data and 1,200 files in the test data.

#### 2. Examine the data

Examine a few files from the training data to get a better sense of what you're working with. What do the images look like? Are there clear distinctions between each class from a visual inspection?

The images show low-resolution squares, circles, and crosses that are of black color on white backgrounds. From a visual inspection, it is easy to differentiate between each shape

#### 3. Set up dataframes

Create two DataFrame objects, one for the training data and one for the test data. For each, have two columns - one with the filenames for the respective dataset and one with the labels (the two letter shape abbreviation) for the respective dataset. How many of each label are in the training set? In the test set? Are the proportions similar?

In [1]:
import os
import pandas as pd

In [20]:
train_dir_path = './squares-circles/train'
test_dir_path = './squares-circles/validation'

test_data_filesnames = pd.Series(os.listdir(test_dir_path))
train_data_filesnames = pd.Series(os.listdir(train_dir_path))

train_data_labels = train_data_filesnames.str[:2]
test_data_labels = test_data_filesnames.str[:2]

train_df = pd.DataFrame({'filename': train_data_filesnames, 'label': train_data_labels})
test_df= pd.DataFrame({'filename': test_data_filesnames, 'label': test_data_labels})

train_df.head(), test_df.head()

(     filename label
 0  cr5543.jpg    cr
 1  ci4352.jpg    ci
 2  cr5225.jpg    cr
 3  cr4891.jpg    cr
 4  ci5058.jpg    ci,
      filename label
 0  sq5857.jpg    sq
 1  ci1570.jpg    ci
 2  sq0091.jpg    sq
 3  sq3598.jpg    sq
 4  cr4113.jpg    cr)

### Part 2: Run code

You're provided with a neural network below that achieves over poor results on the training and test data. The model is pretty simple - it relies on a single densely connected hidden layer. You're going to edit the model to improve it.

#### 1. Compile model

Examine the code below and compile the model. Describe the model. Exactly how many parameters are there? (this should be listed after you compile).

In [13]:
from tensorflow import keras
from tensorflow.keras import initializers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization, LeakyReLU
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [14]:
model=Sequential()

model.add(keras.Input(shape=(32, 32, 1)))
model.add(Flatten())
model.add(Dense(20, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 20)             │        20,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │            63 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,563 (80.32 KB)

 Trainable params: 20,563 (80.32 KB)

 Non-trainable params: 0 (0.00 B)

Our model consists of an input layer which takes in a 32x32 greyscale image (1 channel), a flattening layer to convert it to 1D, a dense hidden layer with 20 nodes, and a dense output layer with 3 nodes. The model has 20,563 parameters.

#### 2. Fix generator

The code below provides you with a keras [ImageDataGenerator](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) object (note that the ImageDataGenerator object in keras has been recently deprecated but it should still work for our purposes). This ImageDataGenerator object takes in an image and rescales the values. Then, `flow_from_dataframe` is called which takes in a DataFrame of files and labels (much like the DataFrame you created) and passes them on. For this step, get the `subset_generator` variable working below by replacing the DataFrame and path references ('train_df' and 'train_folder', respectively). Once running correctly, you should see a message like: _Found 4800 validated image filenames belonging to 3 classes._

In [15]:
subset_generator = ImageDataGenerator(rescale=1/255).flow_from_dataframe(
    train_df,
    train_dir_path,
    x_col='filename', y_col='label',
    target_size=(32, 32),
    class_mode='categorical',
    color_mode="grayscale",
    shuffle=True,
    batch_size = 64)

Found 4800 validated image filenames belonging to 3 classes.


#### 3. Create generator

Next, create a `test_generator` variable that is identical to the `train_generator` above but references the test data (be sure to change the DataFrame variable and the path). Also set the `shuffle` argument to False.

In [16]:
test_generator = ImageDataGenerator(rescale=1/255).flow_from_dataframe(
    test_df,
    test_dir_path,
    x_col='filename', y_col='label',
    target_size=(32, 32),
    class_mode='categorical',
    color_mode="grayscale",
    shuffle=False,
    batch_size = 64)

Found 1200 validated image filenames belonging to 3 classes.


#### 4. Fit model

Fit the model provided on the training data for 12 epochs using the `train_generator` object.

In [18]:
model.fit(subset_generator, epochs = 12)

Epoch 1/12


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.3449 - loss: 1.3310
Epoch 2/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4424 - loss: 1.0717
Epoch 3/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4961 - loss: 1.0180
Epoch 4/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5590 - loss: 0.9505
Epoch 5/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5823 - loss: 0.9099
Epoch 6/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6072 - loss: 0.8673
Epoch 7/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6709 - loss: 0.7975
Epoch 8/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6529 - loss: 0.7810
Epoch 9/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6674 - loss: 0.7449
Epoch 10/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7022 - loss: 0.7132
Epoch 11/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6923 - loss: 0.6927
Epoch 12/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7001 - loss: 0.6765

#### 5. Evaluate model

Use the `.evaluate` call on your trained model with the `train_generator` and `test_generator` objects. What is the performance of the trained model on the training and test data?

In [24]:
model.evaluate(subset_generator)

75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7500 - loss: 0.6143


[0.6194034814834595, 0.7483333349227905]

In [25]:
model.evaluate(test_generator)

12/19 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7353 - loss: 0.6187

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7397 - loss: 0.6186


[0.6185486912727356, 0.7483333349227905]

The training data has an accuracy of about 0.75 while the test data has an accuracy of about 0.7397.

### Part 3: Modify model

Now we'll modify the provided model to see if we can still get strong results using something more simplistic.

#### 1. Edit model

Change the provided model so as to expand the network while improving accuracy. Here are some rough steps to guide you:
- Start by adding a convolutional layer right after the input (between the input layer and the flattening layer) followed by a ReLU (or LeakyReLU) layer.
- Then add a normalization layer and a pooling layer.
- From there, add a dropout layer or another round of convolutions - it's your choice.
- Finally, flatten before connecting to a dense layer which will then connect to your output layer (much like the given model already does).

If you'd like to add a ReLU, normalization, or dropout layer between the dense layers, feel free to do so. Also feel free to change the size/shape of the layers already provided. Take your time with this and experiment to see what works and doesn't. You'll also have to determine the size of any layer you add as well as the kernel/pool size for convolutions and pooling. Fit the model for 12 epochs as you're experimenting and evaluate how it does on your training set using the `train_generator`.

It's quite easy to improve upon the given model and with a bit of experimentation you can get a model that is over 99% accurate (that's not a requirement here). Once you have a model you're comfortable with, move on to the next step.

In [ ]:
model=Sequential()

model.add(keras.Input(shape=(32, 32, 1)))
model.add(Conv2D(32, (3, 3), activation='relu')) # round 1
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu')) # round 2
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(20, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)               │ (None, 30, 30, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 30, 30, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 30, 30, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 13, 13, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 13, 13, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 6, 6, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 20)             │        23,060 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │            63 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,947 (128.70 KB)

 Trainable params: 32,819 (128.20 KB)

 Non-trainable params: 128 (512.00 B)

In [33]:
model.fit(subset_generator, epochs = 12)

Epoch 1/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.7392 - loss: 0.5971
Epoch 2/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9411 - loss: 0.1515
Epoch 3/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9817 - loss: 0.0602
Epoch 4/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9900 - loss: 0.0302
Epoch 5/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9926 - loss: 0.0230
Epoch 6/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9962 - loss: 0.0136
Epoch 7/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9985 - loss: 0.0053
Epoch 8/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9992 - loss: 0.0040
Epoch 9/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9984 - loss: 0.0047
Epoch 10/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9980 - loss: 0.0081
Epoch 11/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9995 - loss: 0.0017
Epoch 12/12
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy:

#### 2. Evaluate model

Use the `.evaluate` call on your updated model with the `test_generator` object. What is the performance of the trained model on the test data?

In [34]:
model.evaluate(test_generator)

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9507 - loss: 0.1191


[0.1178688332438469, 0.9583333134651184]

The accuracy of my final model on the test data is around 0.95

#### 3. Describe your process

What steps did you take to improve the model? What did you try and what seemed to work/not work? How many parameters did your final model include?

To improve the model, I first tried adding one round of convolutions, leakyReLu, normalization, and max pooling. This resulted in my model giving my test data an accuracy around 0.41. I then decided to add another round with similar layers to see if it would help. Everything I did seemed to improve the model's accuracy, so I can say that it worked. This resulted in my model giving my test data an accuracy around 0.95. My final model has 32,947 parameters.